In [4]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
import PyPDF2
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import fitz
import pickle
import os

os.makedirs("additional data", exist_ok=True)
os.makedirs("additional data/links", exist_ok=True)
os.makedirs("additional data/last-data", exist_ok=True)
os.makedirs("additional data/clean_links", exist_ok=True)
os.makedirs("additional data/sub_links", exist_ok=True)



df=pd.read_csv('Untitled spreadsheet - Sheet1.csv')
df

,country,link,pages
0,egypt,https://www.egypte.campusfrance.org/en/recherc...,9
1,tunisie,https://www.tunisie.campusfrance.org/recherche...,50
2,libanon,https://www.liban.campusfrance.org/recherche?p...,53
3,moroco,https://www.maroc.campusfrance.org/recherche?p...,53


### Loop on number of pages and save it in <span style="color:rgb(145, 46, 54); font-size: 25px;">links/country_name.txt</span>


In [5]:
for i in range(4):
    all_links=[]
    for j in range(df['pages'].iloc[i]):
        all_links.append(df['link'].iloc[i]+str(j))
    country=df['country'].iloc[i]
    with open(f'additional data/links/{country}_links.txt', 'w') as file:
        for link in all_links:
            file.write(f"{link}\n")

### Loop on links and extract all links of Articles and save it in <span style="color:rgb(145, 46, 54); font-size: 25px;">sub_links/country_name.txt</span>


In [6]:
countries=['moroco','tunisie','egypt','libanon']

def scrape_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except:
        return None

for country in countries:
    sub_links=[]
    with open(f"additional data/links/{country}_links.txt", 'r') as file:
        my_list = [line.strip() for line in file]


    for i in my_list:
        html_code=scrape_html(i)
        soup = BeautifulSoup(html_code, "html.parser")

        # Find the parent div
        view_content = soup.find("div", class_="view-content")

        # Extract all 'href' links inside divs with class 'views-row'
        for div in view_content.find_all("div", class_="views-row"):
            for a in div.find_all("a", href=True):
                    
                sub_links.append(a["href"])
    
    with open(f'additional data/sub_links/{country}.txt', 'w') as file:
        for link in sub_links:
            file.write(f"{link}\n")


### Loop on all links and clean it from unwanted links like:("/en","/") and save it in <span style="color:rgb(145, 46, 54); font-size: 25px;">clean_links/country_name.txt</span>


In [7]:
countries=["egypt","libanon","moroco","tunisie"]

egypt_domain="https://www.egypte.campusfrance.org"
libanon_domain="https://www.liban.campusfrance.org"
moroco_domain="https://www.maroc.campusfrance.org"
tunisie_domain="https://www.tunisie.campusfrance.org"

domains=[egypt_domain,libanon_domain,moroco_domain,tunisie_domain]

for i in range(4):
    last_links=[]
    with open(f'additional data/sub_links/{countries[i]}.txt', 'r') as file:
        my_list = [line.strip() for line in file]

    for link in my_list:
        if link =='/en' or link =='/':
            my_list.remove(link)

    for link in my_list:
        if "https" not in link:
            last_links.append(domains[i]+link)
        if "https" in link:
            last_links.append(link)
        

    with open(f'additional data/clean_links/{countries[i]}.txt', 'w') as file:
        for link in last_links:
            file.write(f"{link}\n")


# Scrapper

## We have 3 scenarios:
- ##### first scenario: The article only have text
- ##### second scenario: The article have text and pdf file
- ##### third scenario: The link of article is pdf file

## Solutions:
- ##### first scenario: normal scrapping
- ##### second scenario: scrapping the text ; if ".pdf" founded -> extract text from the file and return (content_text+pdf_text)
- ##### third scenario: extract text from the file


### Loop on all links and scrap the content from it and save it in <span style="color:rgb(145, 46, 54); font-size: 25px;">last-data/country_name.txt</span>


In [8]:
paths=["egypt","libanon","moroco","tunisie"]


def extract_text_from_pdf(pdf_url:str) :
        """
        Extracts text from a PDF file located at the provided URL.

        Args:
            pdf_url (str): The URL of the PDF file to extract text from.

        Returns:
            str: The extracted text from the PDF, or None if the PDF cannot be retrieved.

        Raises:
            requests.exceptions.RequestException: If there is an issue with the HTTP request.
            PyMuPDF.PdfDocumentError: If the PDF cannot be opened or processed.
        """

        response = requests.get(pdf_url)
        
        if response.status_code == 200:
            pdf_file = fitz.open(stream=response.content, filetype="pdf")
            
            text = ""
            for page_num in range(pdf_file.page_count):
                page = pdf_file.load_page(page_num)
                text += page.get_text()
            
            return text+"\n"
        else:
            print(f"Failed to retrieve the PDF. Status code: {response.status_code}")
            return None




def scrape_html(url):
    """
    Scrapes the HTML content from the provided URL.

    Args:
        url (str): The URL of the web page to scrape.

    Returns:
        str: The HTML content of the page, or None if the request fails.

    Raises:
        requests.exceptions.RequestException: If there is an issue with the HTTP request.
"""

    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except:
        return None

for path in paths:
    data=[]

    with open(f"additional data/clean_links/{path}.txt", 'r') as file:
        my_list = [line.strip() for line in file]

    iteration=0

    for i in my_list:
        iteration+=1
        pdf_text=""


        if ".pdf" in i :
            pdf_text+=extract_text_from_pdf(i)

        else:
            try:
                html_code=scrape_html(i)
                soup = BeautifulSoup(html_code, 'html.parser')
            except:
                continue

            # Extract content from the div with class 'content'
            try:
                content_text = soup.find('div', class_='content').get_text()
            except:content_text=''
            # Check if any link contains '.pdf'
            pdf_links = soup.find_all('a', href=re.compile(r'\.pdf$', re.IGNORECASE))

            
            if pdf_links:
                for pdf_url in pdf_links:
                    try:
                        pdf_text += extract_text_from_pdf(pdf_url['href'])
                    except:pdf_text+=""

            
        data.append(content_text+"\n \n"+pdf_text)
        print(iteration)

    with open(f"additional data/last-data/{path}.pkl", "wb") as file:
        pickle.dump(data, file)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
Failed to retrieve the PDF. Status code: 404
Failed to retrieve the PDF. Status code: 404
94
95
96
97
98
99
Failed to retrieve the PDF. Status code: 403
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
Failed to retrieve the PDF. Status code: 403
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
1

### Create Documents and Vdb

In [21]:
paths=["egypt","libanon","moroco","tunisie"]
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
os.makedirs("additional data/docs", exist_ok=True)

for path in paths:
    documents = []

    with open(f"additional data/last-data/{path}.pkl", "rb") as file:
        docs = pickle.load(file)

    with open(f"additional data/clean_links/{path}.txt", "rb") as f:
        links = [line.strip() for line in f]


    for link,doc in zip(links,docs):

        if doc !="\n \n":
            documents.append(
                Document(metadata={"source": str(link)}, page_content=doc)
            )

    with open(f"additional data/docs/{path}.pkl", "wb") as f:
        pickle.dump(documents, f)
    vectorstore = Chroma(persist_directory=f'{path}/Vdb-1',embedding_function=embeddings).add_documents(documents);


In [19]:
with open(f"additional data/docs/libanon.pkl", "rb") as file:
    docs = pickle.load(file)
docs

[Document(metadata={'source': "b'https://www.liban.campusfrance.org/zoom-sur-les-etudes-de-medecine'"}, page_content="\n\xa0Intéressés(ées) par les études de médecine en France ?\n\n\n \n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Consultez le document ci-dessous pour plus d'information !\n\n\n\nSur le même sujet\n\n\n\nDocument\n\n\n\n\n\n\nLes études de médecine\n\nFichier Pdf\n1.26 Mo\n\n\n\n\n\n\n\n\nSuivez les grandes étapes pour venir étudier en France\nDécouvrir\n\n\n \n13/12/2024\n1\nLes études \nmédicales en France\nFormation Espace Campus France Liban\nJean-Luc Ito-Pagès\nService Valorisation en l’Enseignement Supérieur et de la Recherche\n2024\nT I T R E  D E  L A  P R E S E N T A T I O N\nLes voies \nd’accès \nMMOPK\nM = Médecine   \nM = Maiëutique\nO = Odontologie\nP = Pharmacie   \nK = Kinésithérapie \nSource graphique : \nCours Galien\n13/12/2024\n2\nT I T R E  D E  L A  P R E S E N T A T I O N\nLes voies \nd’accès \nMMOPK\nM = Mé

In [11]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
persist_directory = 'Visa/Vdb'

vectorstore = Chroma(embedding_function=embeddings, persist_directory=persist_directory)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [15]:
query=""
retriever.get_relevant_documents(query)

[Document(metadata={'source': 'tunisie'}, page_content='\nEn un seul clic, retrouvez nos actualités, nos événements et toutes autres informations dédiées aux études en France.\n\n\n \n\n\nImage\n \n\n\n\n\n\n\n\n\n\nSuivez-nous sans plus tarder sur Instagram\n@ campus_france_tunisie\nhttps://www.instagram.com/campus_france_tunisie/?hl=fr\n\n\n\n\n \n'),
 Document(metadata={'source': 'tunisie'}, page_content='\nÀ l’occasion de la Journée internationale des Femmes (8 mars), l’Institut français de Sfax, le Club Inner Wheel et la librairie Culturel sont heureux d’accueillir Amel Djait, journaliste et experte en communication.\n\n\n \nElle viendra nous raconter son parcours et sa passion de ses "1001 Tunisie", le jeudi 8 mars à 17h à la Libraire Culturel.\nFondatrice du guide et du magazine de voyage et de lifestyle 1001 Tunisie, Amel Djait a reçu différents prix, notamment celui de "l’Innovation Google Tunisie" en juin 2013. En 2017, elle est primée, en tant qu’auteur, par l’Académie Natio